In [12]:
import os

# Tell huggingface_hub / transformers to use the mirror
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

# Optional but recommended for faster downloads
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

import torch
import transformers
import accelerate
import bitsandbytes
import safetensors

print("torch version:", torch.__version__)
print("transformers version:", transformers.__version__)
print("accelerate version:", accelerate.__version__)
print("bitsandbytes version:", bitsandbytes.__version__)
print("safetensors version:", safetensors.__version__)

print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device count:", torch.cuda.device_count())
    print("Current device:", torch.cuda.current_device())
    print("Device name:", torch.cuda.get_device_name(0))


torch version: 2.1.2+cu118
transformers version: 4.37.2
accelerate version: 0.21.0
bitsandbytes version: 0.45.5
safetensors version: 0.6.2
CUDA available: True
CUDA device count: 1
Current device: 0
Device name: Tesla V100S-PCIE-32GB


In [13]:
import torch
from PIL import Image
import os

from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path, tokenizer_image_token
from llava.conversation import conv_templates, SeparatorStyle

In [14]:
model_path = "liuhaotian/llava-v1.6-mistral-7b"

device = "cuda"
dtype = torch.float16

tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path=model_path,
    model_base=None,          # base model if using LoRA; else keep None
    model_name=get_model_name_from_path(model_path),
    device=device, 
    torch_dtype=dtype
)

tokenizer.pad_token = tokenizer.eos_token  # if no pad token defined
model.config.pad_token_id = tokenizer.pad_token_id


model.eval()

print("Loaded model:", model_path)
print("Context length:", context_len)

/root/miniconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


/root/miniconda3/lib/python3.10/site-packages/torch/nn/modules/module.py:2025: UserWarning: for vision_model.embeddings.class_embedding: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(f'for {key}: copying from a non-meta parameter in the checkpoint to a meta '
/root/miniconda3/lib/python3.10/site-packages/torch/nn/modules/module.py:2025: UserWarning: for vision_model.embeddings.patch_embedding.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(f'for {key}: copying from a non-meta parameter in the checkpoint to a meta '
/root

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loaded model: liuhaotian/llava-v1.6-mistral-7b
Context length: 2048


In [4]:
import torch
import gc
import numpy as np
from PIL import Image
import types
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN
from llava.mm_utils import process_images, tokenizer_image_token

# 1. Clean Memory
torch.cuda.empty_cache()
gc.collect()

class LLaVAPGDAttacker:

    def __init__(self, model, tokenizer, image_processor, device="cuda"):
        self.model = model
        self.tokenizer = tokenizer
        self.image_processor = image_processor
        self.device = device
        self.dtype = model.dtype

        # --- STEP 1: GENERAL SETUP ---
        self.model.train()
        
        if hasattr(self.model, "gradient_checkpointing_disable"):
            self.model.gradient_checkpointing_disable()

        # --- STEP 2: SURGICAL UNFREEZE ---
        self.model.requires_grad_(False)
        vision_tower = model.get_vision_tower()
            
        vision_tower.requires_grad_(True)
        vision_tower.train() 
        
        if hasattr(self.model.model, "mm_projector"):
            self.model.model.mm_projector.requires_grad_(True)
            self.model.model.mm_projector.train()

        self.mean = torch.tensor(image_processor.image_mean).view(1, 3, 1, 1).to(device).to(self.dtype)
        self.std = torch.tensor(image_processor.image_std).view(1, 3, 1, 1).to(device).to(self.dtype)

    def prepare_inputs(self, prompt, target_response):
        full_text = f"{DEFAULT_IMAGE_TOKEN}\n{prompt} {target_response}"
        input_ids = tokenizer_image_token(
            full_text, self.tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt'
        ).unsqueeze(0).to(self.device)
        
        prompt_text = f"{DEFAULT_IMAGE_TOKEN}\n{prompt} "
        prompt_ids = tokenizer_image_token(
            prompt_text, self.tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt'
        )
        prompt_len = prompt_ids.shape[0]
        
        labels = input_ids.clone()
        labels[:, :prompt_len] = -100 
        return input_ids, labels

    # UPDATED: Added 'prompt' argument with default None
    def attack(self, image_path, target_text, prompt=None, epsilon=8/255, alpha=2/255, num_steps=50):
        self.model.train()
        torch.cuda.empty_cache()
        
        raw_image = Image.open(image_path).convert("RGB")
        h, w = self.image_processor.crop_size['height'], self.image_processor.crop_size['width']
        image_resized = raw_image.resize((w, h), resample=Image.BICUBIC)
        
        img_tensor = torch.tensor(np.array(image_resized)).permute(2, 0, 1).unsqueeze(0).to(self.device)
        img_tensor = (img_tensor / 255.0).to(self.dtype) 
        
        delta = torch.zeros_like(img_tensor).uniform_(-epsilon, epsilon).to(self.device, dtype=self.dtype)
        delta.requires_grad_(True)
        
        # --- LOGIC UPDATE FOR VQA VS CAPTIONING ---
        if prompt is None:
            # Default behavior (Captioning)
            # Hardcoded prompt + "an image of {target}"
            final_prompt = ("USER: Name a single main visual object in this image. "
                "For example, if the object contains a cat, say 'cat'. "
                "Return ONLY the single word for the object.\n"
                "ASSISTANT:")
            final_target = f"an image of {target_text}"
        else:
            # VQA behavior
            # Use specific prompt (Question) + exact target (Answer)
            final_prompt = prompt
            final_target = target_text

        input_ids, labels = self.prepare_inputs(final_prompt, final_target)

        for step in range(num_steps):
            adv_img_01 = torch.clamp(img_tensor + delta, 0, 1)
            adv_img_norm = (adv_img_01 - self.mean) / self.std
            
            outputs = self.model(
                input_ids=input_ids,
                labels=labels,
                images=adv_img_norm, 
                use_cache=False
            )
            
            loss = outputs.loss
            
            if not loss.requires_grad:
                print("FATAL: Loss does not require grad. Surgical unfreeze failed.")
                return None
                
            loss.backward()
            
            if delta.grad is None:
                print(f"FATAL Step {step}: Delta.grad is None.")
                return None

            with torch.no_grad():
                grad = delta.grad.detach()
                delta.data = delta.data - alpha * grad.sign()
                delta.data = torch.clamp(delta.data, -epsilon, epsilon)
                delta.data = torch.clamp(img_tensor + delta.data, 0, 1) - img_tensor
            
            delta.grad.zero_()
            
            if step % 10 == 0:
                print(f"Step {step}/{num_steps} | Loss: {loss.item():.4f}")
                torch.cuda.empty_cache()

        final_adv_img = torch.clamp(img_tensor + delta, 0, 1).detach().float().cpu()
        return Image.fromarray((final_adv_img.squeeze(0).permute(1, 2, 0).numpy() * 255).astype('uint8'))

In [15]:
import types

def forward_with_grad(self, images):
    if type(images) is list:
        image_features = []
        for image in images:
            image_forward_out = self.vision_tower(
                image.to(device=self.device, dtype=self.dtype).unsqueeze(0), 
                output_hidden_states=True
            )
            image_feature = self.feature_select(image_forward_out).to(image.dtype)
            image_features.append(image_feature)
    else:
        # This is the path your PGD attack usually takes
        image_forward_outs = self.vision_tower(
            images.to(device=self.device, dtype=self.dtype), 
            output_hidden_states=True
        )
        image_features = self.feature_select(image_forward_outs).to(images.dtype)
    return image_features

vision_tower = model.get_vision_tower()
vision_tower.forward = types.MethodType(forward_with_grad, vision_tower)

# --- Verification Wrapper ---
def verify_attack(image_path, question):
    # Switch to Eval for deterministic generation
    model.eval()
    prompt = f"{DEFAULT_IMAGE_TOKEN}\n{question}"
    input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).to("cuda")
    
    attention_mask = torch.ones_like(input_ids, device=device)

    image = Image.open(image_path).convert("RGB")
    image_tensor = process_images([image], image_processor, model.config)[0].to(model.device, dtype=torch.float16)

    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            attention_mask=attention_mask,       # <--- Pass the mask here!
            pad_token_id=tokenizer.pad_token_id, # <--- Pass the ID explicitly to silence the second warning
            images=image_tensor.unsqueeze(0),
            image_sizes=[image.size],
            do_sample=False,
            max_new_tokens=128
        )
    return tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0].strip()

In [6]:
import re
import torch
from llava.mm_utils import tokenizer_image_token

def _gt_contains_object(obj_word: str, coco_ground_truths):
    """
    Check if obj_word appears as a WHOLE WORD in any COCO caption.
    """
    pattern = rf"\b{re.escape(obj_word)}\b"
    for gt in coco_ground_truths:
        if re.search(pattern, gt.lower()):
            return True
    return False

def generate_attack_config(image_path, coco_ground_truths, model, tokenizer):
    """
    Generates attack targets without printing debug info.
    Returns dict if valid config found, else None.
    """
    # --- Step 1: Get Clean Object Description ---
    clean_caption = verify_attack(
        image_path,
        f"USER: Name a single main visual object in this image. "
        "For example, if the object contains cat, say 'cat'. "
        "Return ONLY the single word for the main object.\n"
        "ASSISTANT:"
    )
    
    # Basic cleanup
    main_object = clean_caption.lower().replace("'", "").replace('"', "").strip(".,!")
    if not main_object:
        return None
    main_object = main_object.split()[0]

    # --- Step 2: Verify Baseline (Must be correct initially) ---
    is_correct = _gt_contains_object(main_object, coco_ground_truths)
    if not is_correct:
        return None 

    # --- Step 3: Generate Similar Target ---
    prompt = (
        f"USER: Name a visual object that looks similar to a '{main_object}' but is different. "
        "For example, if the object is 'cat', say 'dog'. If 'car', say 'truck'. "
        "Return ONLY the single word for the new object.\n"
        "ASSISTANT:"
    )

    input_ids = tokenizer_image_token(
        prompt, 
        tokenizer, 
        image_token_index=-200, 
        return_tensors='pt'
    ).unsqueeze(0).to("cuda")

    with torch.no_grad():
        output = model.generate(input_ids, max_new_tokens=10, do_sample=False)

    raw_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0].strip().lower()

    # --- Step 4: Clean Target Output ---
    target_object = raw_output.replace("'", "").replace('"', "").strip(".,!")
    if not target_object:
        return None
    target_object = target_object.split()[0]

    # Sanity check: Don't attack 'cat' with 'cat'
    if target_object == main_object:
        return None

    # --- Step 5: Verify Target is NOT in Ground Truth ---
    if _gt_contains_object(target_object, coco_ground_truths):
        return None

    # --- Step 6: Return Valid Config ---
    target_caption = f"A photo of a {target_object}."

    return {
        "original_object": main_object,
        "target_object": target_object,
        "target_caption": target_caption,
        "clean_caption": clean_caption, 
    }

In [7]:
from typing import List, Dict, Any, Optional
from tqdm.auto import tqdm
from pathlib import Path
import json
import random
import os

def build_coco_attack_dataset(
    img_dir: str,
    ann_file: str,
    max_images: Optional[int] = None,
) -> List[Dict[str, Any]]:
    """
    Build an attack dataset using your preprocessed caption JSON + generate_attack_config.

    Expected ann_file format:
    [
      {
        "image_id": 105156,
        "file_name": "COCO_val2014_000000105156.jpg",
        "captions": ["...", "...", ...]
      },
      ...
    ]
    """
    random.seed(42)

    # ----------------------------
    # Load your caption JSON (list of dicts)
    # ----------------------------
    with open(ann_file, "r") as f:
        caption_entries = json.load(f)

    # Shuffle to randomize image order
    random.shuffle(caption_entries)

    attack_examples: List[Dict[str, Any]] = []

    img_dir = str(img_dir)

    for entry in tqdm(caption_entries, desc="Building attack dataset"):
        img_id = entry["image_id"]
        file_name = entry["file_name"]
        coco_gts = entry["captions"]  # list of strings

        image_path = os.path.join(img_dir, file_name)

        # Use your function to decide if this image is attackable
        attack_cfg = generate_attack_config(
            image_path=image_path,
            coco_ground_truths=coco_gts,
            model=model,
            tokenizer=tokenizer,
        )
        if attack_cfg is None:
            continue

        # Store an entry
        attack_examples.append({
            "image_id": img_id,
            "image_path": image_path,
            "coco_ground_truths": coco_gts,
            "original_object": attack_cfg["original_object"],
            "target_object": attack_cfg["target_object"],
            "target_caption": attack_cfg["target_caption"],
        })

        if max_images is not None and len(attack_examples) >= max_images:
            break

    return attack_examples


In [8]:
import json

output_file = "coco_attack_dataset_1000.json"

if os.path.exists(output_file):
    with open(output_file, "r") as f:
        dataset = json.load(f)

else:
    dataset = build_coco_attack_dataset(
        "/root/COCO/image/", 
        ann_file='/root/COCO/caption/captions.json', 
        max_images=1000
    )
    
    # Save the dataset for next time
    with open(output_file, "w") as f:
        json.dump(dataset, f, indent=4)
        
    print(f"Dataset generated and saved to {output_file}")

In [11]:
import pandas as pd
import os
from tqdm import tqdm

# --- Configuration ---
RESULTS_DIR = "./results/adversarial_bench_v1"
IMAGES_DIR = os.path.join(RESULTS_DIR, "images")
CSV_PATH = os.path.join(RESULTS_DIR, "attack_log.csv")

os.makedirs(IMAGES_DIR, exist_ok=True)
attacker = LLaVAPGDAttacker(model, tokenizer, image_processor, device="cuda")
# Initialize global log
results_log = []

print(f"Starting evaluation on {len(dataset)} samples...")
count = 0
for sample in tqdm(dataset[:50]):
    image_id = sample['image_id']
    target_text = sample['target_caption']
    target_obj = sample['target_object']
    orig_obj = sample['original_object']
    
    # 1. Run PGD Attack
    adv_pil = attacker.attack(
        image_path=sample['image_path'],
        target_text=target_obj,
        epsilon=16/255, 
        num_steps=50    
    )
    
    if adv_pil is None:
        print(f"Attack failed for Image {image_id}")
        continue
        
    # 2. Save Logic (The Fix)
    if count < 10:
        # Permanent save for analysis
        save_filename = f"{image_id}_{orig_obj}_to_{target_obj}.png"
        save_path = os.path.join(IMAGES_DIR, save_filename)
        adv_pil.save(save_path)
        count += 1
    else:
        # Temporary save for verification (Overwrites each time)
        # CRITICAL: We update save_path so verify_attack sees the NEW image
        save_path = "temp_verification_image.png"
        adv_pil.save(save_path)
    
    # 3. Verify
    prompt = (f"USER: Name a single main visual object in this image. "
        "For example, if the object contains a cat, say 'cat'. "
        "Return ONLY the single word for the object.\n"
        "ASSISTANT:")
        
    generated_caption = verify_attack(
        save_path,
        prompt
    )
    
    if not generated_caption:
        print(f"Verification failed for Image {image_id}")
        continue
    # 4. Calculate Metrics
    gen_lower = generated_caption.lower()
    
    # TSR (Targeted Success Rate): Did we force the model to say "chair"?
    # We check word boundaries to avoid partial matches (e.g. 'cat' in 'catch')
    tsr_hit = bool(re.search(rf"\b{re.escape(target_obj)}\b", gen_lower))
    
    # USR (Untargeted Success Rate): Did we force the model to STOP saying "table"?
    usr_hit = not bool(re.search(rf"\b{re.escape(orig_obj)}\b", gen_lower))
    
    # 5. Log Data
    results_log.append({
        "image_id": image_id,
        "original_object": orig_obj,
        "target_object": target_obj,
        "target_caption": target_text,
        "generated_caption": generated_caption,
        "targeted_success": tsr_hit,   # Success: It said the target
        "untargeted_success": usr_hit, # Success: It didn't say the original
        "epsilon": 16/255,
        "steps": 50,
        "image_path": save_path
    })

# --- Final Analysis ---
df = pd.DataFrame(results_log)

if not df.empty:
    print("\n" + "="*40)
    print(f"Total Samples: {len(df)}")
    print(f"Targeted Success Rate (TSR):   {df['targeted_success'].mean() * 100:.2f}%")
    print(f"Untargeted Success Rate (USR): {df['untargeted_success'].mean() * 100:.2f}%")
    print("="*40)

    # Save to CSV
    df.to_csv(CSV_PATH, index=False)
    print(f"Results saved to: {CSV_PATH}")
else:
    print("No results generated.")

Starting evaluation on 720 samples...


  0%|          | 0/50 [00:00<?, ?it/s]

Step 0/50 | Loss: 4.1676
Step 10/50 | Loss: 1.8360
Step 20/50 | Loss: 0.7317
Step 30/50 | Loss: 0.4955
Step 40/50 | Loss: 0.2367


  2%|▏         | 1/50 [00:18<15:14, 18.66s/it]

Step 0/50 | Loss: 3.5870
Step 10/50 | Loss: 1.3377
Step 20/50 | Loss: 0.7754
Step 30/50 | Loss: 0.6686
Step 40/50 | Loss: 0.3134


...

Step 0/50 | Loss: 5.7809
Step 10/50 | Loss: 2.5691
Step 20/50 | Loss: 1.3947
Step 30/50 | Loss: 0.8759
Step 40/50 | Loss: 0.8872


 98%|█████████▊| 49/50 [15:24<00:18, 18.93s/it]

Step 0/50 | Loss: 5.1326
Step 10/50 | Loss: 2.6069
Step 20/50 | Loss: 1.2952
Step 30/50 | Loss: 0.8837
Step 40/50 | Loss: 0.6625


100%|██████████| 50/50 [15:42<00:00, 18.86s/it]


Total Samples: 50
Targeted Success Rate (TSR):   64.00%
Untargeted Success Rate (USR): 82.00%
Results saved to: ./results/adversarial_bench_v1/attack_log.csv


In [14]:
def parse_custom_vqa_data(questions_data, annotations_data, image_dir):
    """
    Parses your specific JSON format where file_name is provided.
    questions_data: List of question dicts
    annotations_data: List of answer dicts
    """
    # Map Question ID to Answers
    qid_to_answers = {}
    for item in annotations_data:
        # Collect all valid answers
        answers = [ans['answer'] for ans in item['answers']]
        qid_to_answers[item['question_id']] = answers

    dataset = []
    
    for q_item in questions_data:
        qid = q_item['question_id']
        
        # Skip if no answers found
        if qid not in qid_to_answers:
            continue
            
        # Use the explicit file_name provided in your JSON
        image_path = os.path.join(image_dir, q_item['file_name'])
        
        if not os.path.exists(image_path):
            print(f"Warning: Image not found at {image_path}")
            continue

        dataset.append({
            "question_id": qid,
            "image_path": image_path,
            "question": q_item['question'],
            "gt_answers": qid_to_answers[qid] # List of valid answers
        })
        
    print(f"Loaded {len(dataset)} samples ready for attack.")
    return dataset

In [45]:
import re
import torch
from llava.mm_utils import tokenizer_image_token

def check_fuzzy_match(prediction, targets):
    """
    Checks if the ESSENTIAL words from the target appear in the prediction.
    Robust to:
    - Stopwords ("the", "a", "is")
    - Punctuation
    - Pluralization (horses <-> horse) via simple stemming
    """
    if pd.isna(prediction) or pd.isna(targets):
        return False
        
    # Ensure targets is a list
    if isinstance(targets, str):
        targets = [targets]
        
    stopwords = {"a", "an", "the", "is", "are", "to", "of", "in", "on", "with", "for", "'s", "job"}
    
    def normalize(text):
        # 1. Lowercase and remove punctuation
        text = re.sub(r'[^\w\s]', ' ', str(text).lower())
        
        tokens = []
        for w in text.split():
            # 2. Filter stopwords
            if w in stopwords: continue
            
            # 3. Simple Stemming: Remove trailing 's' 
            # (Fixes "horses" != "horse")
            if w.endswith('s') and len(w) > 3: 
                w = w[:-1]
                
            tokens.append(w)
        return tokens

    pred_tokens = normalize(prediction)
    pred_text = " ".join(pred_tokens)
    
    for gt in targets:
        gt_tokens = normalize(gt)
        if not gt_tokens: continue
        
        # Check 1: Substring match of normalized string
        gt_text = " ".join(gt_tokens)
        if gt_text in pred_text:
            return True
            
        # Check 2: Bag-of-words intersection (>50% match)
        # Allows for word order changes or extra words in prediction
        matches = sum(1 for w in gt_tokens if any(w in p for p in pred_tokens))
        if matches >= len(gt_tokens):
            return True
            
    return False

def generate_vqa_attack_config(sample, model, tokenizer):
    image_path = sample['image_path']
    question = sample['question']
    gt_answers = sample['gt_answers']
    
    # 1. Baseline: Ask for a complete sentence
    prompt_str = f"USER: {question}\nBased on given image, answer the question using a single complete sentence.\nASSISTANT:"
    
    clean_pred = verify_attack(image_path, prompt_str).strip().replace("\n", " ")
    
    # 2. Robust Validation
    is_correct = check_fuzzy_match(clean_pred, gt_answers)
                 
    if not is_correct:
        # Debug print to see why it failed (can remove later)
        print(f"Skipping: Pred '{clean_pred}' vs GT {gt_answers}")
        return None 
        
    # Treat the Model's OWN sentence as the Ground Truth for the attack
    gt_sentence = clean_pred

    # 3. Generate Target: Ask for a False Sentence
    gen_prompt = (
        f"USER: The question is: '{question}'. The correct answer is: '{gt_sentence}'. "
        "Provide a plausible but INCORRECT answer sentence that describes the image differently. The answer must be relative to the image context but wrong."
        "For example, if the answer is 'The man is drinking water', say 'The man is eating an apple'. "
        "Return ONLY the incorrect sentence.\n"
        "ASSISTANT:"
    )
    
    input_ids = tokenizer_image_token(gen_prompt, tokenizer, -200, return_tensors='pt').unsqueeze(0).to("cuda")
    
    with torch.no_grad():
        output = model.generate(input_ids, max_new_tokens=40, do_sample=False)
        
    target_answer = tokenizer.batch_decode(output, skip_special_tokens=True)[0].strip().replace("\n", " ")
    
    # 4. Final Cleanup & Sanity Check
    # Remove quotes if model added them
    target_answer = target_answer.replace('"', '').replace("'", "")
    
    # Ensure target isn't identical to original
    if target_answer.lower() == gt_sentence.lower():
        print("Skipping: Target identical to GT.")
        return None
        
    # Ensure target doesn't accidentally contain the real answer keywords
    # (e.g. "The horse is NOT pulling a cart" -> contains "pulling cart")
    if check_fuzzy_match(target_answer, gt_answers):
        # We want the target to be WRONG, so if it matches GT, we skip it.
        print("Skipping: Target accidentally matches GT.")
        return None

    return {
        "prompt": prompt_str,
        "clean_pred": gt_sentence,
        "target": target_answer,
        "gt_answers": gt_answers
    }

In [39]:
def load_custom_vqa_data(q_path, a_path, img_dir, max_samples=1000):
    print("Loading VQA Data...")
    with open(q_path, 'r') as f: q_data = json.load(f)
    # Handle list vs dict format
    questions = q_data.get('questions', q_data) if isinstance(q_data, dict) else q_data
    
    with open(a_path, 'r') as f: a_data = json.load(f)
    annotations = a_data.get('annotations', a_data) if isinstance(a_data, dict) else a_data

    # Map QID -> Answers
    qid_to_ans = {}
    for item in annotations:
        qid_to_ans[item['question_id']] = [ans['answer'] for ans in item['answers']]
    
    dataset = []
    # indices = list(range(len(questions)))
    # random.shuffle(indices) # Optional Shuffle
    
    for q in questions:
        if len(dataset) >= max_samples: break
        
        qid = q['question_id']
        if qid not in qid_to_ans: continue
            
        # Use 'file_name' if available, otherwise construct standard COCO format
        if 'file_name' in q:
            fname = q['file_name']
        else:
            fname = f"{q['image_id']:012d}.jpg"
            
        img_path = os.path.join(img_dir, fname)
        if os.path.exists(img_path):
            dataset.append({
                "question_id": qid,
                "image_path": img_path,
                "question": q['question'],
                "gt_answers": qid_to_ans[qid]
            })
            
    print(f"Loaded {len(dataset)} samples.")
    return dataset

In [ ]:
QUESTIONS_FILE = "/root/COCO/questions/questions.json"
ANSWERS_FILE = "/root/COCO/answers/answers.json"
IMAGE_DIR = "/root/COCO/image/"

# 2. Logging
RESULTS_DIR = "/root/AMDM/results/vqa_experiment"
CSV_PATH = RESULTS_DIR + "/vqa_attack_log.csv"
MAX_IMAGES_TO_SAVE = 10  # Only save 10 examples to disk

os.makedirs(RESULTS_DIR, exist_ok=True)

attacker = LLaVAPGDAttacker(model, tokenizer, image_processor)
vqa_dataset = load_custom_vqa_data(QUESTIONS_FILE, ANSWERS_FILE, IMAGE_DIR, max_samples=1000)

results_log = []
saved_count = 0

print("\n--- Starting Evaluation Loop ---")
for sample in tqdm(vqa_dataset):
    # 1. Config
    config = generate_vqa_attack_config(sample, model, tokenizer)
    if config is None: continue
    
    # 2. Attack
    adv_img = attacker.attack(
        image_path=sample['image_path'],
        target_text=config['target'], 
        prompt=config['prompt'], 
        epsilon=16/255,
        num_steps=50
    )
    
    if adv_img is None: continue
    
    # 3. Save Logic
    if saved_count < MAX_IMAGES_TO_SAVE:
        save_path = os.path.join(RESULTS_DIR, f"{sample['question_id']}_adv.png")
        adv_img.save(save_path)
        verify_path = save_path
        saved_count += 1
    else:
        verify_path = "temp_eval_image.png"
        adv_img.save(verify_path)

    # 4. Verify & Score
    final_pred = verify_attack(verify_path, config['prompt']).strip().replace("\n", " ")
    
    # Targeted Success: Did it say the specific lie?
    success_targeted = check_fuzzy_match(final_pred, [config['target']])
    
    # Untargeted Success / Drop: Is it NO LONGER correct?
    is_still_correct = check_fuzzy_match(final_pred, sample['gt_answers'])
    success_untargeted = not is_still_correct
    
    results_log.append({
        "question_id": sample['question_id'],
        "question": sample['question'],
        "target_forced": config['target'],
        "final_pred": final_pred,
        "targeted_success": success_targeted,
        "accuracy_drop": success_untargeted
    })

# 5. Report & Save
if results_log:
    df = pd.DataFrame(results_log)
    print("\n" + "="*40)
    print(f"Total Evaluated: {len(df)}")
    print(f"Targeted Success Rate (ASR):   {df['targeted_success'].mean()*100:.2f}%")
    print(f"Accuracy Drop (Untargeted):    {df['accuracy_drop'].mean()*100:.2f}%")
    print("="*40)
    df.to_csv(CSV_PATH, index=False)
    print(f"Log saved to {CSV_PATH}")


Loading VQA Data...
Loaded 2 samples.

--- Starting Evaluation Loop ---


  0%|          | 0/2 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Step 0/50 | Loss: 3.0855
Step 10/50 | Loss: 0.9074
Step 20/50 | Loss: 0.8631
Step 30/50 | Loss: 0.7712
Step 40/50 | Loss: 0.8064


100%|██████████| 2/2 [00:22<00:00, 11.13s/it]

Skipping: Pred 'The pizza has 8 slices.' vs GT ['9', '9', 'several', '10', '10', '10', '10', '10 left', '10', '10']

Total Evaluated: 1
Targeted Success Rate (ASR):   0.00%
Accuracy Drop (Untargeted):    100.00%
Log saved to /root/AMDM/results/vqa_experiment/vqa_attack_log.csv
